In [39]:
import dgl
from collections import OrderedDict
from random import randint
import numpy as np

def get_fan(cur_layer):
    if (cur_layer == 1):
        return 10
    return 25

def simulate(khop, roots, graph, v2idx, partition=4):
    
    # The simulator assume the aggregation occurs in the gpu that stores the parent node's features
    # all leaf nodes aggregate locally
    # send local aggregate results to parent node's gpu if needed
    
    nodes_set = [roots]
    edges_pair_set = []
    
    total_edge_pair = 0 # total number of edges in the sample graph (including all layers)
    total_cross_edge = 0 # number of edges cross the partition
    total_memcpy = 0 # number of aggregated data transfer
    
    total_cross_workload = np.zeros((partition, ), dtype=int) # global aggregation workload
    total_local_workload = np.zeros((partition, ), dtype=int) # local aggregation workload

    print(f"-----Start Sampling-----")
    for cur_layer in range(1, khop+1):
        sample_nodes = nodes_set[cur_layer - 1]
        fan = get_fan(cur_layer)
        graph_sample = dgl.sampling.sample_neighbors(graph, sample_nodes, fanout=fan, edge_dir="out")
        v_set, e_set = graph_sample.edges()
        _next = list(OrderedDict.fromkeys(e_set)) # remove duplicates
        nodes_set.append(_next)
        edges_pair_set.append([v_set, e_set])
        
    print(f"-----Start Simulating-----")
    for cur_layer in range(0, khop):
        print(f"-----Layer{cur_layer}-----")
        v_set, e_set = edges_pair_set[cur_layer]
        layer_cross_workload = np.zeros((partition, ), dtype=int) # global aggregation workload
        layer_local_workload = np.zeros((partition, ), dtype=int) # local aggregation workload


        size_v = len(v_set)
        total_edge_pair += size_v

        pos = [0] * (size_v + 1)
        idx = 0
        prev = -1
        for i in range(0, size_v):
            cur_parent_id = int(v_set[i])
            if cur_parent_id != prev:
                prev = cur_parent_id
                idx = idx + 1
                pos[idx] = i
        idx += 1
        pos[idx] = size_v
            
        for i in range(0, idx):
            start = pos[i]
            end = pos[i+1]
            parent_id = int(v_set[start])
            parent_pa = v2idx[parent_id]
            local_workload = np.zeros((partition, ), dtype=int) # current edge pairs' local aggregation workload            

            cross_dest = v2idx[parent_id] # which GPU to send the locally aggregated results


            for j in range(start, end):
                leaf_id = int(e_set[j])
                leaf_pt = v2idx[leaf_id]
                local_workload[leaf_pt] += 1
            
            for i in range(0, partition):
                if i != cross_dest and local_workload[i] > 0:
                    layer_cross_workload[cross_dest] += 1
                if i != parent_pa:
                    total_cross_edge += local_workload[i]
            
            np.sum([local_workload, layer_local_workload], axis = 0, out = layer_local_workload)   
        
        np.sum([total_local_workload, layer_local_workload], axis = 0, out = total_local_workload)
        np.sum([total_cross_workload, layer_cross_workload], axis = 0, out = total_cross_workload)
        layer_workload = np.sum([layer_local_workload, layer_cross_workload], axis=0)
        layer_sum = np.sum(layer_workload)
        print("local workload: ", layer_local_workload)
        print("cross workload: ", layer_cross_workload)
        print("total workload: ",  layer_workload)
        print("layer distribution: ",  np.divide(layer_workload, layer_sum))


    total_workload = np.sum([total_local_workload, total_cross_workload], axis=0)
    total_aggregation = np.sum(total_workload)
    total_memcpy = np.sum(total_cross_workload)
    print("batch distribution: ", np.divide(total_workload, total_aggregation))
    return total_edge_pair, total_cross_edge, total_memcpy

In [40]:
from random import sample
import numpy as np

# ------ Configuration ------
# dataset = dgl.data.CoraGraphDataset()
# dataset = dgl.data.CiteseerGraphDataset()
dataset = dgl.data.PubmedGraphDataset()
partition_num = 4
khop = 4
roots_size = 15
num_samples = 10
## Start 

graph = dataset[0] 
graph = dgl.remove_self_loop(graph)
graph = dgl.add_self_loop(graph)
print(f"e_num={graph.num_edges()}; v_num={graph.num_nodes()}")
v2idx = dgl.metis_partition_assignment(graph, partition_num)

v_num = graph.num_nodes()
v_list = [i for i in range(0, v_num)]

total_e_num = 0
total_c_num = 0
total_k_num = 0

for i in range(1, num_samples+1):
    print(f"batch: {i}")
    roots = sample(v_list, roots_size)
    e_num, c_num, k_num = simulate(khop=khop, roots=roots, graph=graph, v2idx=v2idx)
    total_e_num += e_num
    total_c_num += c_num
    total_k_num += k_num
    print("\n")
    
print(f"e_num={total_e_num}; c_num={total_c_num}; k_num={total_k_num}")
if total_k_num > 0:
    print(f"cache miss:  {total_c_num * 100.0 / total_e_num }%")
    print(f"memory copy: {total_k_num * 100.0 / total_e_num }%")

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
e_num=108365; v_num=19717
Convert a graph into a bidirected graph: 0.002 seconds
Metis partitioning: 0.024 seconds
batch: 1
-----Start Sampling-----
-----Start Simulating-----
-----Layer0-----
local workload:  [ 6 33 15 14]
cross workload:  [0 3 1 1]
total workload:  [ 6 36 16 15]
layer distribution:  [0.08219178 0.49315068 0.21917808 0.20547945]
-----Layer1-----
local workload:  [ 97 249 172 131]
cross workload:  [ 5 17  9  3]
total workload:  [102 266 181 134]
layer distribution:  [0.14934114 0.38945827 0.26500732 0.19619327]
-----Layer2-----
local workload:  [1202 1980 1803 1721]
cross workload:  [ 67 130 115  73]
total workload:  [1269 2110 1918 1794]
layer distribution:  [0.17895924 0.29756029 0.27048371 0.25299676]
-----Layer3-----
local workload:  [15973 20486 20992 23957]
cross workload:  [ 899 1340 